<a href="https://colab.research.google.com/github/Dipak22/Case-Studies/blob/master/Attention_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

batch_size = 4
seq_len = 64
embed_dim = 128

x = torch.randn(batch_size, seq_len, embed_dim)
print("X shape: ", x.shape)

X shape:  torch.Size([4, 64, 128])


In [7]:
# temporary implemetation

similarity = x @ x.transpose(1,2)
print("similarilty shape: ", similarity.shape)
print("Pre normalized variance:", similarity.var())
similarity_norm = similarity/(embed_dim**0.5)
print("Normalized variance: ", similarity_norm.var())

attn = similarity_norm.softmax(dim=-1)

context_vectors = attn@x
print("output shape: ", context_vectors.shape)

similarilty shape:  torch.Size([4, 64, 64])
Pre normalized variance: tensor(387.0992)
Normalized variance:  tensor(3.0242)
output shape:  torch.Size([4, 64, 128])


In [8]:
#Self Attention
class Attention(nn.Module):
  def __init__(self, embed_dim):
    super().__init__()

    self.embed_dim = embed_dim
    self.query = nn.Linear(self.embed_dim, self.embed_dim)
    self.key = nn.Linear(self.embed_dim, self.embed_dim)
    self.value = nn.Linear(self.embed_dim, self.embed_dim)
  def forward(self,x):
    q = self.query(x)
    k = self.key(x)
    v = self.value(x)
    similarity = (q @ k.transpose(1,2))/(self.embed_dim**0.5)
    attention = similarity.softmax(dim = -1)
    output = attention @ v
    return output

attention = Attention(embed_dim=128)
output = attention(x)
print(output.shape)


torch.Size([4, 64, 128])
